In [1]:
import os
import torch
from model_lightning_seg import MyModel
import yaml
from addict import Dict

# === CONFIGURA ESTAS RUTAS ===
ckpt_dir = "/Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/experiment_5_giancarlo"
config_path = "default_config_train_seg.yaml"
onnx_out_dir = "./onnx_models"
os.makedirs(onnx_out_dir, exist_ok=True)

# === LEE CONFIG ===
with open(config_path, "r") as f:
    conf = Dict(yaml.safe_load(f))

# === LISTA CHECKPOINTS ===
ckpts = sorted([os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir) if f.endswith(".ckpt")])

# === EXPORTA CADA MODELO ===
for i, ckpt_path in enumerate(ckpts):
    print(f"📦 Exportando modelo {i+1}/{len(ckpts)} → {ckpt_path}")
    
    # Carga el modelo Lightning
    model_lightning = MyModel.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        model_opts=conf.model_opts,
        train_par=conf.train_par,
        strict=False,
    )
    model_lightning.eval()
    
    # Extrae el modelo real (UNet)
    #model = model_lightning.model
    #model.eval()

    # Dummy input
    dummy_input = torch.randn(1, 1, 128, 128, 128, dtype=torch.float32)

    # Ruta de salida
    onnx_path = os.path.join(onnx_out_dir, f"model_fold{i}.onnx")

    # Exporta a ONNX
    torch.onnx.export(
        model_lightning,
        dummy_input,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
        export_params=True,
        opset_version=17,
    )
    print(f"✅ Guardado en: {onnx_path}\n")

print("🎉 ¡Conversión completa!")


📦 Exportando modelo 1/10 → /Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_1.ckpt


/Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/src/models_arq/att_unet_seg.py:45: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x_up.shape[2:] != y.shape[2:]:
/Users/emilio/anaconda3/envs/lightning/lib/python3.12/site-packages/torch/onnx/symbolic_helper.py:1454: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.


OnnxExporterError: Module onnx is not installed!